In [281]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [282]:
data = pd.read_csv('./DataWithGDP.csv')

countries = list(np.unique(data.loc[:, 'Region, subregion, country or area *']))

reg_data = data[['Region, subregion, country or area *', 'Year', 'GDP per capita_x']]

years = set(np.arange(1950, 2022))

In [283]:
removed_countries = []
full_df = []
for country in countries:
    country_df = reg_data[reg_data['Region, subregion, country or area *'] == country]
    country_df = country_df[country_df['GDP per capita_x'] != -1]
    if not country_df.empty:
        x = country_df.iloc[:, 1]
        unknown = np.array(list(years - set(x))).reshape(-1, 1)
        unknown.sort()
        x = x.values.reshape(-1, 1)
        y = country_df.iloc[:, 2].values.reshape(-1, 1)
        model = LinearRegression()
        model.fit(x, y)
        pred = model.predict(unknown)
        pred_df = pd.DataFrame({'Region, subregion, country or area *': country,
                                'Year': unknown.reshape(-1, ), 'GDP per capita_x': pred.reshape(-1, )})
        full_df.append(pd.concat([country_df, pred_df], ignore_index=True))
    else:
        removed_countries.append(country)
        data.drop(data[data['Region, subregion, country or area *'] == country].index, inplace=True)
full_gdp = pd.concat(full_df, ignore_index=True)

In [284]:
full_gdp.sort_values(['Region, subregion, country or area *', 'Year'], inplace=True)

In [277]:
# remain_countries = list(set(countries) - set(removed_countries))
# remain_countries.sort()
# for i in remain_countries:
#     plt.scatter(list(years),
#                 full_gdp[full_gdp['Region, subregion, country or area *'] == i]
#                 .loc[:, 'GDP per capita_x'])
#     plt.show()

In [285]:
data.drop(columns=['GDP per capita_x', 'GDP per capita_y'], inplace=True)

In [292]:
data_gdp = pd.merge(data, full_gdp, on=['Region, subregion, country or area *', 'Year'])
data_gdp.rename(columns={'GDP per capita_x': 'GDP'}, inplace=True)

In [293]:
data_gdp.to_csv('./DataWithFullGDP.csv')